In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize']=(8,6)
mpl.rcParams['axes.grid']=False

from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.metrics import mean_absolute_percentage_error

from keras.layers import *
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

In [13]:
df=pd.read_csv('/kaggle/input/medianhouseprice/MSPUS.csv')

In [18]:
rows,columns=df.shape[0],df.shape[1]
print(rows,columns)

In [26]:
#df['DATE']=pd.to_datetime(df['DATE'])
#df.set_index('DATE',inplace=True)

In [14]:
df.head()

In [25]:
df.describe().transpose()

In [29]:
import matplotlib.dates as mdates

fig,ax=plt.subplots(figsize=(30,8))
monthly_locator=mdates.MonthLocator()
year_locator=mdates.MonthLocator(interval=12)
year_month_formatter=mdates.DateFormatter("%Y-%m")
ax.xaxis.set_major_locator(year_locator)
ax.xaxis.set_minor_locator(monthly_locator)
ax.xaxis.set_major_formatter(year_month_formatter)
ax.plot(df.index,df.MSPUS)
fig.autofmt_xdate()

In [30]:
# Check stationarity of the data
def adf_test(timeseries):
    print("Results of Augmented Dickey-fuller test")
    dftest = adfuller(timeseries,autolag='AIC')
    dfoutput=pd.Series(dftest[0:4],index=['Test Statistic','p-value','#lags used','number of observations used'])
    for key,value in dftest[4].items():
        dfoutput['critical value (%s)'%key]= value
    print(dfoutput)

In [31]:
df['first_diff']=df['MSPUS'].diff(periods=1)
df.head()

In [34]:
adf_test(df.first_diff.dropna())

P-Value is at the border . Its less than 0.05 but not much so that we can decide about wether the series is stationary or not.

In [35]:
# Check KPSS test
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','#Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

In [36]:
kpss_test(df.first_diff.dropna())

Pvalue is not >0.05 so it seems series is not stationary.

In [37]:
# Check ACF(Autocorrelation)
fig,ax=plt.subplots(figsize=(14,6))
plot_acf(df.first_diff.dropna(),ax=ax,lags=100);
plt.ylim([-0.5,1.05]);

Since series does not pass both tests, we can say that its not stationary.